Final Project : MultiThreaded Web Server
---

It’s been a long journey, but we’ve reached the end of the book. In this  
chapter, we’ll build one more project together to demonstrate some of the  
concepts we covered in the final chapters, as well as recap some earlier  
lessons.

For our final project, we’ll make a web server that says “hello” and looks  
like Figure 20-1 in a web browser.

![image](https://doc.rust-lang.org/book/img/trpl20-01.png)

> Figure 20-1: Our final shared project

Here is our plan for building the web server:

1. Learn a bit about TCP and HTTP.
2. Listen for TCP connections on a socket.
3. Parse a small number of HTTP requests.
4. Create a proper HTTP response.
5. Improve the throughput of our server with a thread pool.

Before we get started, we should mention one detail:  

- the method we’ll use won’t be the best way to build a web server with Rust.  

Community members have published a number of production-ready crates available  
on crates.io that provide more complete web server and thread pool  
implementations than we’ll build. However, our intention in this chapter is to  
help you learn, not to take the easy route. Because Rust is a systems  
programming language, we can choose the level of abstraction we want to work  
with and can go to a lower level than is possible or practical in other  
languages. We’ll therefore write the basic HTTP server and thread pool manually  
so you can learn the general ideas and techniques behind the crates you might  
use in the future.

### Building a Single-Threaded Web Server

We’ll start by getting a single-threaded web server working. Before we begin,  
let’s look at a quick overview of the protocols involved in building web  
servers. The details of these protocols are beyond the scope of this book, but  
a brief overview will give you the information you need.

The two main protocols involved in web servers are Hypertext Transfer Protocol  
(HTTP) and Transmission Control Protocol (TCP). Both protocols are  
request-response protocols, meaning a client initiates requests and a server  
listens to the requests and provides a response to the client. The contents of  
those requests and responses are defined by the protocols.

TCP is the lower-level protocol that describes the details of how information  
gets from one server to another but doesn’t specify what that information is.  
HTTP builds on top of TCP by defining the contents of the requests and  
responses. It’s technically possible to use HTTP with other protocols, but in  
the vast majority of cases, HTTP sends its data over TCP. We’ll work with the  
raw bytes of TCP and HTTP requests and responses.

### Listening to the TCP Connection

Our web server needs to listen to a TCP connection, so that’s the first part  
we’ll work on. The standard library offers a `std::net` module that lets us do  
this. Let’s make a new project in the usual fashion:

```sh
$ cargo new hello
     Created binary (application) `hello` project
$ cd hello
```

Now enter the code in Listing 20-1 in src/main.rs to start. This code will  
listen at the local address 127.0.0.1:7878 for incoming TCP streams. When it  
gets an incoming stream, it will print Connection established!.

> Filename: src/main.rs

In [ ]:
use std::net::TcpListener;

{
    // binding the specific spot in our computer's memory where the listener
    // is setup
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        println!("Connection established!");
    }
}


> Listing 20-1: Listening for incoming streams and printing a message when we receive a stream

Using TcpListener, we can listen for TCP connections at the address  
127.0.0.1:7878. In the address, the section before the colon is an IP address  
representing your computer (this is the same on every computer and doesn’t  
represent the authors’ computer specifically), and 7878 is the port. We’ve  
chosen this port for two reasons:  

- HTTP isn’t normally accepted on this port so our server is unlikely to  
conflict with any other web server you might have running on your machine, and  
- 7878 is rust typed on a telephone.

The bind function in this scenario works like the new function in that it will  
return a new TcpListener instance. The function is called bind because, in  
networking, connecting to a port to listen to is known as “binding to a port.”

The bind function returns a Result<T, E>, which indicates that it’s possible  
for binding to fail. For example, connecting to port 80 requires administrator  
privileges (nonadministrators can listen only on ports higher than 1023), so if  
we tried to connect to port 80 without being an administrator, binding wouldn’t  
work. Binding also wouldn’t work, for example, if we ran two instances of our  
program and so had two programs listening to the same port. Because we’re  
writing a basic server just for learning purposes, we won’t worry about  
handling these kinds of errors; instead, we use unwrap to stop the program if  
errors happen.

The incoming method on TcpListener returns an iterator that gives us a sequence  
of streams (more specifically, streams of type TcpStream). A single stream  
represents an open connection between the client and the server. A connection  
is the name for the full request and response process in which a client  
connects to the server, the server generates a response, and the server closes  
the connection. As such, we will read from the TcpStream to see what the client  
sent and then write our response to the stream to send data back to the client.  
Overall, this for loop will process each connection in turn and produce a  
series of streams for us to handle.

For now, our handling of the stream consists of calling unwrap to terminate our  
program if the stream has any errors; if there aren’t any errors, the program  
prints a message. We’ll add more functionality for the success case in the next  
listing. The reason we might receive errors from the incoming method when a  
client connects to the server is that we’re not actually iterating over  
connections. Instead, we’re iterating over connection attempts. The connection  
might not be successful for a number of reasons, many of them operating system  
specific. For example, many operating systems have a limit to the number of  
simultaneous open connections they can support; new connection attempts beyond  
that number will produce an error until some of the open connections are closed.

Let’s try running this code! Invoke cargo run in the terminal and then  
load 127.0.0.1:7878 in a web browser. The browser should show an error message  
like “Connection reset,” because the server isn’t currently sending back any  
data. But when you look at your terminal, you should see several messages that  
were printed when the browser connected to the server!

```sh
Running `target/debug/hello`
Connection established!
Connection established!
Connection established!
```

Sometimes, you’ll see multiple messages printed for one browser request; the  
reason might be that the browser is making a request for the page as well as a  
request for other resources, like the favicon.ico icon that appears in the  
browser tab.

It could also be that the browser is trying to connect to the server multiple  
times because the server isn’t responding with any data. When stream goes out  
of scope and is dropped at the end of the loop, the connection is closed as  
part of the drop implementation. Browsers sometimes deal with closed  
connections by retrying, because the problem might be temporary. The important  
factor is that we’ve successfully gotten a handle to a TCP connection!

Remember to stop the program by pressing ctrl-c when you’re done running a  
particular version of the code. Then restart the program by invoking the cargo  
run command after you’ve made each set of code changes to make sure you’re  
running the newest code.

### Reading the Request

Let’s implement the functionality to read the request from the browser! To  
separate the concerns of first getting a connection and then taking some action  
with the connection, we’ll start a new function for processing connections. In  
this new handle_connection function, we’ll read data from the TCP stream and  
print it so we can see the data being sent from the browser. Change the code to  
look like Listing 20-2.

> Filename: src/main.rs

In [ ]:
use std::{
    io::{prelude::*, BufReader},
    net::{TcpListener, TcpStream},
}

fn handle_connection(mut stream: TcpStream){
    let buf_reader = BufReader::new(&mut stream);
    let http_request: Vec<_> = buf_reader
        .lines()
        .map(|result| result.unwrap())
        .take_while(|line| !line.is_empty())
        .collect();

    println!("Request: {:#?}", http_request);
}

{
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        handle_connection(stream);
    }
}

> Listing 20-2: Reading from the TcpStream and printing the data

We bring std::io::prelude and std::io::BufReader into scope to get access to  
traits and types that let us read from and write to the stream. In the for loop  
in the main function, instead of printing a message that says we made a  
connection, we now call the new handle_connection function and pass the stream  
to it.

In the handle_connection function, we create a new BufReader instance that  
wraps a mutable reference to the stream. BufReader adds buffering by managing  
calls to the std::io::Read trait methods for us.

We create a variable named http_request to collect the lines of the request the  
browser sends to our server. We indicate that we want to collect these lines in  
a vector by adding the Vec<_> type annotation.

BufReader implements the std::io::BufRead trait, which provides the lines  
method. The lines method returns an iterator of Result<String, std::io::Error>  
by splitting the stream of data whenever it sees a newline byte. To get each  
String, we map and unwrap each Result. The Result might be an error if the data  
isn’t valid UTF-8 or if there was a problem reading from the stream. Again, a  
production program should handle these errors more gracefully, but we’re  
choosing to stop the program in the error case for simplicity.

The browser signals the end of an HTTP request by sending two newline  
characters in a row, so to get one request from the stream, we take lines until  
we get a line that is the empty string. Once we’ve collected the lines into the  
vector, we’re printing them out using pretty debug formatting so we can take a  
look at the instructions the web browser is sending to our server.

Let’s try this code! Start the program and make a request in a web browser again. Note that we’ll still get an error page in the browser, but our program’s output in the terminal will now look similar to this:

```sh
$ cargo run
   Compiling hello v0.1.0 (file:///projects/hello)
    Finished dev [unoptimized + debuginfo] target(s) in 0.42s
     Running `target/debug/hello`
Request: [
    "GET / HTTP/1.1",
    "Host: 127.0.0.1:7878",
    "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:99.0) Gecko/20100101 Firefox/99.0",
    "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language: en-US,en;q=0.5",
    "Accept-Encoding: gzip, deflate, br",
    "DNT: 1",
    "Connection: keep-alive",
    "Upgrade-Insecure-Requests: 1",
    "Sec-Fetch-Dest: document",
    "Sec-Fetch-Mode: navigate",
    "Sec-Fetch-Site: none",
    "Sec-Fetch-User: ?1",
    "Cache-Control: max-age=0",
]
```

Depending on your browser, you might get slightly different output. Now that  
we’re printing the request data, we can see why we get multiple connections  
from one browser request by looking at the path after GET in the first line of  
the request. If the repeated connections are all requesting /, we know the  
browser is trying to fetch / repeatedly because it’s not getting a response  
from our program.

Let’s break down this request data to understand what the browser is asking of  
our program.

### A Closer Look at an HTTP Request

HTTP is a text-based protocol, and a request takes this format:

```sh
Method Request-URI HTTP-Version CRLF
headers CRLF
message-body
```

The first line is the request line that holds information about what the client  
is requesting. The first part of the request line indicates the method being  
used, such as GET or POST, which describes how the client is making this  
request. Our client used a GET request, which means it is asking for  
information.

The next part of the request line is /, which indicates the Uniform Resource  
Identifier (URI) the client is requesting: a URI is almost, but not quite, the  
same as a Uniform Resource Locator (URL). The difference between URIs and URLs  
isn’t important for our purposes in this chapter, but the HTTP spec uses the  
term URI, so we can just mentally substitute URL for URI here.

The last part is the HTTP version the client uses, and then the request line  
ends in a CRLF sequence. (CRLF stands for carriage return and line feed, which  
are terms from the typewriter days!) The CRLF sequence can also be written as  
\r\n, where \r is a carriage return and \n is a line feed. The CRLF sequence  
separates the request line from the rest of the request data. Note that when  
the CRLF is printed, we see a new line start rather than \r\n.

Looking at the request line data we received from running our program so far,  
we see that :  

- `GET` is the `method`,  
- `/` is the `request URI`,  
- and `HTTP/1.1` is the `version`.

After the request line, the remaining lines starting from Host:

- onward are headers.  
- GET requests have no body.

Try making a request from a different browser or asking for a different  
address, such as `127.0.0.1:7878/test`, to see how the request data changes.

Now that we know what the browser is asking for, let’s send back some data!

### Writing a Response

We’re going to implement sending data in response to a client request.  
Responses have the following format:

```sh
HTTP-Version Status-Code Reason-Phrase CRLF
headers CRLF
message-body
```

The first line is :  

- a status line that contains the HTTP version used in the response,  
- a numeric status code that summarizes the result of the request, and  
- a reason phrase that provides a text description of the status code.  

After the CRLF sequence are any headers, another CRLF sequence, and the body of  
the response.

Here is an example response that :  

- uses HTTP version 1.1,  
- has a status code of 200,  
- an OK reason phrase,  
- no headers, and  
- no body:

```sh
HTTP/1.1 200 OK\r\n\r\n
```

- The status code `200` is the `standard success` response.  
- The text is a tiny successful `HTTP` response.  

Let’s write this to the stream as our response to a successful request! From  
the handle_connection function, remove the println! that was printing the  
request data and replace it with the code in Listing 20-3.

> Filename: src/main.rs

In [2]:
use std::{
    // - prelude::* brings in the IO prelude, which includes traits like Read  
    // and Write
    // - BufReader is a buffered reader type, which can read from a source and  
    // buffer the input
    io::{prelude::*, BufReader},
    net::{TcpListener, TcpStream}, //  types for TCP network communication
}

/// Takes a mutable TcpStream as a parameter
/// TcpStream represents a TCP connection to a remote end point
fn handle_connection_v2(mut stream: TcpStream) {
    // BufReader object is created from the TcpStream
    // This will allow reading from the stream in a buffered fashion for  
    // efficiency and convenience
    let buf_reader = BufReader::new(&mut stream);
    // reads HTTP request lines from the buffered reader
    // Vec<_> is a placeholder type, which tells the compiler to infer the
    // type of the Vec
    let http_request: Vec<_> = buf_reader
        // lines() generates an iterator over the lines in the stream
        .lines()
        // map() unwraps each line (panicking if there's an error)
        .map(|result| result.unwrap())
        // take_while() stops the iteration when it encounters an empty line  
        // (which marks the end of the HTTP headers)
        .take_while(|line| !line.is_empty())
        // collect() collects the resulting lines into a Vec
        .collect();

    println!("Request: {:#?}", http_request);
    // simple HTTP response string is created 
    // This is a minimal "200 OK" response, meaning that the request was  
    // successful
    let response = "HTTP/1.1 200 OK\r\n\r\n";

    // - writes the response string back to the client
    // - as_bytes() converts the string into bytes that can be written to the  
    // stream
    // - unwrap() panics if there's an error
    stream.write(response.as_bytes()).unwrap();
}

{
    // creates a TCP listener that listens for incoming connections on the  
    // local IP address 127.0.0.1 and port 7878
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    // Iterates over the incoming connections to the TCP listener
    // Each incoming connection is a TcpStream that can be read from and  
    // written to
    for stream in listener.incoming() {
        // unwrap() is used to panic if there's an error accepting a connection
        let stream = stream.unwrap();
        // handle each incoming connection
        handle_connection_v2(stream);
    }
}


> Listing 20-3: Writing a tiny successful HTTP response to the stream

- The first new line defines the response variable that holds the success  
message’s data. 
- Then we call `as_bytes` on our response to convert the string data to bytes.  
- The `write_all` method on stream takes a `&[u8]` and sends those bytes  
directly down the connection.  
- Because the `write_all` operation could fail, we use `unwrap` on any  
`error result` as before.  
- Again, in a real application you would add `error handling` here.

With these changes, let’s run our code and make a request. We’re no longer  
printing any data to the terminal, so we won’t see any output other than the  
output from Cargo. When you load `127.0.0.1:7878` in a web browser :  

- you should get a blank page  
- instead of an error.  

You’ve just hand-coded receiving an HTTP request and sending a response!

Let’s implement the functionality for returning more than a blank page. Create  
the new file hello.html in the root of your project directory, not in the src  
directory. You can input any HTML you want; Listing 20-4 shows one possibility.

> Filename: hello.html

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Hello!</title>
  </head>
  <body>
    <h1>Hello!</h1>
    <p>Hi from Rust</p>
  </body>
</html>
```

Listing 20-4: A sample HTML file to return in a response

This is a minimal HTML5 document with a heading and some text. To return this  
from the server when a request is received, we’ll modify handle_connection as  
shown in Listing 20-5 to read the HTML file, add it to the response as a body,  
and send it.

> Filename: src/main.rs

In [2]:
use std::{
    fs,
    io::{prelude::*, BufReader},
    net::{TcpListener, TcpStream},
}

fn handle_connection_html(mut stream: TcpStream) {
    let buf_reader = BufReader::new(&mut stream);
    let http_request: Vec<_> = buf_reader
        .lines()
        .map(|result| result.unwrap())
        .take_while(|line| !line.is_empty())
        .collect();

    let status_line = "HTTP/1.1 200 OK";
    // We created an HTML file called hello.html at root dir and are loading it
    let contents = fs::read_to_string("hello.html").unwrap();
    let length = contents.len();

    let response = format!(
        "{status_line}\r\nContent-Length: {length}\r\n\r\n{contents}"
    );

    stream.write_all(response.as_bytes()).unwrap();
}

{
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        handle_connection_html(stream);
    }
}

> Listing 20-5: Sending the contents of hello.html as the body of the response

We’ve added fs to the use statement to bring the standard library’s filesystem  
module into scope. The code for reading the contents of a file to a string  
should look familiar; we used it in Chapter 12 when we read the contents of a  
file for our I/O project in Listing 12-4.

Next, we use format! to add the file’s contents as the body of the success  
response. To ensure a valid HTTP response, we add the Content-Length header  
which is set to the size of our response body, in this case the size of 
`hello.html`.

Run this code with `cargo run` and load `127.0.0.1:7878` in your browser; you  
should see your HTML rendered!

Currently, we’re ignoring the request data in `http_request` and just sending  
back the contents of the HTML file unconditionally. That means if you try  
requesting `127.0.0.1:7878/something-else` in your browser, you’ll still get  
back this same HTML response. At the moment, our server is very limited and  
does not do what most web servers do. We want to customize our responses  
depending on the request and only send back the HTML file for a well-formed  
request to /.

### Validating the Request and Selectively Responding

Right now, our web server will return the HTML in the file no matter what the  
client requested. Let’s add functionality to check that the browser is  
requesting / before returning the HTML file and return an error if the browser  
requests anything else. For this we need to modify handle_connection, as shown  
in Listing 20-6. This new code checks the content of the request received  
against what we know a request for / looks like and adds if and else blocks to  
treat requests differently.

> Filename: src/main.rs

In [2]:
use std::{
    fs,
    io::{prelude::*, BufReader},
    net::{TcpListener, TcpStream},
}

fn handle_connection_req(mut stream: TcpStream){
    let buf_reader = BufReader::new(&mut stream);

    // We only care about the first line of the request
    // - Calling next to get the first item from the iterator
    // - Unwrapping the result to get the line
    let request_line = buf_reader.lines().next().unwrap().unwrap();

    if request_line == "GET / HTTP/1.1"{
        let status_line = "HTTP/1.1 200 OK";
        let contents = fs::read_to_string("hello.html").unwrap();
        let length = contents.len();

        let response = format!(
            "{status_line}\r\nContent-Length: {length}\r\n\r\n(contents)"
        );

        stream.write_all(response.as_bytes()).unwrap();
    }
    else {
        // some other request
    }
}

{
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        handle_connection_req(stream);
    }
}

> Listing 20-6: Handling requests to / differently from other requests

We’re only going to be looking at the first line of the HTTP request, so rather  
than reading the entire request into a vector :  

- we’re calling `next` to get the first item from the iterator.  
- The first unwrap takes care of the Option and stops the program if the  
iterator has no items.  
- The second unwrap handles the Result and has the same effect as the unwrap  
that was in the map added in Listing 20-2.

Next, we check the request_line to see if it equals the request line of a GET  
request to the / path. If it does, the if block returns the contents of our  
HTML file.

If the request_line does not equal the GET request to the / path, it means  
we’ve received some other request. We’ll add code to the else block in a moment  
to respond to all other requests.

Run this code now and request `127.0.0.1:7878`;  

- you should get the HTML in `hello.html`.  
- If you make any other request, such as `127.0.0.1:7878/something-else`,  
- you’ll get a connection error like those you saw when running the code in  
Listing 20-1 and Listing 20-2.

Now let’s add the code in Listing 20-7 to the else block to return a response  
with the status code 404, which signals that the content for the request was  
not found. We’ll also return some HTML for a page to render in the browser  
indicating the response to the end user.

> Filename: src/main.rs

In [2]:
use std::{
    fs,
    io::{prelude::*, BufReader},
    net::{TcpListener, TcpStream},
}

fn handle_connection_status(mut stream: TcpStream) {
    let buf_reader = BufReader::new(&mut stream);

    let request_line = buf_reader.lines().next().unwrap().unwrap();

    if request_line == "GET / HTTP/1.1"{
        let status_line = "HTTP/1.1 200 OK";
        let contents = fs::read_to_string("hello.html").unwrap();
        let length = contents.len();

        let response = format!(
            "{status_line}\r\nContent-Length: {length}\r\n\r\n{contents}"
        );

        stream.write_all(response.as_bytes()).unwrap();
    }
    else{
        let status_line = "HTTP/1.1 404 NOT FOUND";
        let contents = fs::read_to_string("404.html").unwrap();
        let length = contents.len();

        let response = format!(
            "{status_line}\r\nContent-Length: {length}\r\n\r\n{contents}"
        );

        stream.write_all(response.as_bytes()).unwrap();
    }
}

{
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        handle_connection_status(stream);
    }
}

> Listing 20-7: Responding with status code 404 and an error page if anything  
other than / was requested

Here, our response has a status line with status code 404 and the reason phrase  
NOT FOUND. The body of the response will be the HTML in the file 404.html.  
You’ll need to create a 404.html file next to hello.html for the error page;  
again feel free to use any HTML you want or use the example HTML in  
Listing 20-8.

> Filename: 404.html

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Hello!</title>
  </head>
  <body>
    <h1>Oops!</h1>
    <p>Sorry, I don't know what you're asking for.</p>
  </body>
</html>
```

> Listing 20-8: Sample content for the page to send back with any 404 response

With these changes, run your server again. Requesting 127.0.0.1:7878 should  
return the contents of hello.html, and any other request,  
like 127.0.0.1:7878/foo, should return the error HTML from 404.html.

### A Touch of Refactoring

At the moment the if and else blocks have a lot of repetition: they’re both  
reading files and writing the contents of the files to the stream. The only  
differences are the status line and the filename. Let’s make the code more  
concise by pulling out those differences into separate if and else lines that  
will assign the values of the status line and the filename to variables; we can  
then use those variables unconditionally in the code to read the file and write  
the response. Listing 20-9 shows the resulting code after replacing the large  
if and else blocks.

> Filename: src/main.rs

In [4]:
use std::{
    fs,
    io::{prelude::*, BufReader},
    net::{TcpListener, TcpStream},
}

fn handle_connection_status_v2(mut stream: TcpStream) {
    let buf_reader = BufReader::new(&mut stream);

    let request_line = buf_reader.lines().next().unwrap().unwrap();

    let (status_line, filename) = if request_line == "GET / HTTP/1.1" {
        ("HTTP/1.1 200 OK", "hello.html")
    }
    else {
        ("HTTP/1.1 404 NOT FOUND", "404.html")
    };

    let contents = fs::read_to_string(filename).unwrap();
    let length = contents.len();

    let response =
        format!(
            "{status_line}\r\nContent-Length: {length}\r\n\r\n{contents}"
        );
    
    stream.write_all(response.as_bytes()).unwrap();
}

{
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        handle_connection_status_v2(stream);
    }
}

> Listing 20-9: Refactoring the if and else blocks to contain only the code  
that differs between the two cases

Now the if and else blocks only return the appropriate values for the status  
line and filename in a tuple; we then use destructuring to assign these two  
values to status_line and filename using a pattern in the let statement, as  
discussed in Chapter 18.

The previously duplicated code is now outside the if and else blocks and uses  
the status_line and filename variables. This makes it easier to see the  
difference between the two cases, and it means we have only one place to update  
the code if we want to change how the file reading and response writing work.  
The behavior of the code in Listing 20-9 will be the same as that in  
Listing 20-8.

Awesome! We now have a simple web server in approximately 40 lines of Rust code  
that :  

- responds to one request with a `page of content` and  
- responds to all other requests with a `404 response`

Currently, our server runs in a single thread, meaning it can only serve one  
request at a time. Let’s examine how that can be a problem by simulating some  
slow requests. Then we’ll fix it so our server can handle multiple requests at  
once.

Turning Our Single-Threaded Server into a Multithreaded Server
---